In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
package_dir = "../input/pretrained-models.pytorch-master/"
sys.path.insert(0, package_dir)
import pretrainedmodels
package_dir = '../input/early-stopping-pytorch'
sys.path.append(package_dir)
from pytorchtools import EarlyStopping
import numpy as np
import pandas as pd
import scipy as sp
from functools import partial
from sklearn import metrics
from sklearn.model_selection import KFold
from collections import Counter
import json
import time
import gc
import torchvision
import torch.nn as nn
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device=device)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# To have reproducible results and compare them
nr_seed = 2019
import numpy as np 
np.random.seed(nr_seed)

# Specify GPU usage
device_no = 4
torch.cuda.set_device(device_no)
device = torch.device(f"cuda:{device_no}")

In [ ]:
# 前処理を記述
# 現状はほぼなにもしてない
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


transform_valid = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# dfのimage_idから対応する画像を読み込んだり、前処理をかけたりする関数
class RetinopathyDatasetTrain(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/train_images',
                                self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        return {'image': image,
                'labels': label
                }
    
class RetinopathyDatasetTest(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name =  os.path.join('../input/aptos2019-blindness-detection/test_images',
                                 self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        return {'image': image}
    
## kappa_lossにとる最適化をするclass。使い方勉強中
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
def load_model(pretrained='imagenet'):
    model = pretrainedmodels.__dict__['resnet101'](pretrained=pretrained)
    model.avg_pool = nn.AdaptiveAvgPool2d(1)
    model.last_linear = nn.Sequential(
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.25),
                              nn.Linear(in_features=2048, out_features=2048, bias=True),
                              nn.ReLU(),
                              nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                              nn.Dropout(p=0.5),
                              nn.Linear(in_features=2048, out_features=1, bias=True),
                             )
    return model

In [ ]:
#params
lr = 1e-5
img_size = 224
batch_size = 48
n_epochs = 25
n_freeze = 1
n_folds = 2
patience = 2
coef = [0.5, 1.5, 2.5, 3.5]
_load_state = True # fine-tuning済みのモデルを使うか 

# Data
train_dataset = RetinopathyDatasetTrain(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                        transform=transform)
valid_dataset = RetinopathyDatasetTrain(csv_file='../input/aptos2019-blindness-detection/train.csv', 
                                        transform=transform_valid)

# CV Split
num_train = len(train_dataset)
indices = list(range(num_train))
kf = KFold(n_splits=n_folds, random_state=1337, shuffle=True)

train_idx = []
valid_idx = []

for t, v in kf.split(indices):
    train_idx.append(t)
    valid_idx.append(v)

# Training                        
for fold in np.arange(n_folds):
    since = time.time()
    print('Fold:',fold)
    
    # Model
    if _load_state:
        model = load_model(pretrained=None)
        # deviceを指定しないとcuda:0に勝手に乗っかってしまう
        model.load_state_dict(torch.load("../input/mmmodel/model.bin", map_location=device))
    else:
        model = load_model(pretrained='imagenet')
    model = model.to(device)
    
    criterion = nn.MSELoss()
    plist = [{'params': model.parameters(), 'lr': 1e-5}]
    optimizer = optim.Adam(plist, lr=1e-5)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

    # Train with early stopping
    
    train_sampler = SubsetRandomSampler(train_idx[fold])
    valid_sampler = SubsetRandomSampler(valid_idx[fold])

    data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4, sampler=train_sampler)
    data_loader_valid = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=4, sampler=valid_sampler)
    
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    for epoch in range(n_epochs):
        if epoch == n_freeze:      
            for param in model.parameters():
                param.requires_grad = True
        
        print('Epoch {}/{}'.format(epoch, n_epochs - 1))
        print('-' * 10)
        scheduler.step()
        model.train()
        running_loss = 0.0
        tk0 = tqdm(data_loader_train, total=int(len(data_loader_train)))
        counter = 0
        for bi, d in enumerate(tk0):
            inputs = d["image"]
            labels = d["labels"].view(-1, 1)
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            counter += 1
            tk0.set_postfix(loss=(running_loss / (counter * data_loader_train.batch_size)))
        epoch_loss = running_loss / len(data_loader_train)
        print('Training Loss: {:.4f}'.format(epoch_loss))

        model.eval()
        eval_loss = 0
        nb_eval_steps = 0

        for step, batch in enumerate(tqdm(data_loader_valid , total=int(len(data_loader_valid)))):

            inputs = batch["image"]
            labels = batch["labels"].view(-1, 1)

            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            with torch.no_grad():
                outputs = model(inputs)

            y_hat = torch.Tensor.cpu(outputs.view(-1))
            y = torch.Tensor.cpu(labels.view(-1))

            for pred in enumerate(y_hat):
                if pred[1] < coef[0]:
                    y_hat[1] = 0
                elif pred[1] >= coef[0] and pred[1] < coef[1]:
                    y_hat[1] = 1
                elif pred[1] >= coef[1] and pred[1] < coef[2]:
                    y_hat[1] = 2
                elif pred[1] >= coef[2] and pred[1] < coef[3]:
                    y_hat[1] = 3
                else:
                    y_hat[1] = 4

            tmp_eval_loss = quadratic_kappa(y_hat, y)

            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

        eval_loss = eval_loss / nb_eval_steps

        print('Validation Kappa: {:.4f}'.format(eval_loss))

        eval_loss = 1 - eval_loss
        early_stopping(eval_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
     
    del(model, data_loader_train, data_loader_valid)
    gc.collect()
    torch.cuda.empty_cache() 

#train for save model
cvを使わない本番の学習

In [ ]:
#params
lr = 1e-5
img_size = 224
batch_size = 48
n_epochs = 2
n_freeze = 1
patience = 2
coef = [0.5, 1.5, 2.5, 3.5]
_load_state = True # fine-tuning済みのモデルを使うか 

# Data
train_dataset = RetinopathyDatasetTrain(csv_file="../input/aptos2019-blindness-detection/train.csv",
                                        transform=transform)
data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4)

since = time.time()
# Model
if _load_state:
    model = load_model(pretrained=None)
    # deviceを指定しないとcuda:0に勝手に乗っかってしまう
    model.load_state_dict(torch.load("../input/mmmodel/model.bin", map_location=device))
else:
    model = load_model(pretrained='imagenet')
model = model.to(device)

criterion = nn.MSELoss()
plist = [{'params': model.parameters(), 'lr': 1e-5}]
optimizer = optim.Adam(plist, lr=1e-5)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

for epoch in range(n_epochs):
    if epoch == n_freeze:      
        for param in model.parameters():
            param.requires_grad = True

    print('Epoch {}/{}'.format(epoch, n_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(data_loader_train, total=int(len(data_loader_train)))
    counter = 0
    for bi, d in enumerate(tk0):
        inputs = d["image"]
        labels = d["labels"].view(-1, 1)
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * data_loader_train.batch_size)))
    epoch_loss = running_loss / len(data_loader_train)
    print('Training Loss: {:.4f}'.format(epoch_loss))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "../input/mmmodel/model_0726_01.bin")

del(model, data_loader_train)
gc.collect()
torch.cuda.empty_cache() 

#Inference

In [ ]:
model = load_model(pretrained=None)
model.load_state_dict(torch.load("../input/mmmodel/model_0726_01.bin", map_location=device))
model.eval()
model = model.to(device)

In [ ]:
test_dataset = RetinopathyDatasetTest(csv_file='../input/aptos2019-blindness-detection/sample_submission.csv',
                                      transform=transform)

In [ ]:
test_bs = 16
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_bs, shuffle=False, num_workers=4)
test_preds = np.zeros((len(test_dataset), 1))
tk0 = tqdm(test_data_loader)
for i, x_batch in tqdm(enumerate(tk0)):
    x_batch = x_batch["image"]
    pred = model(x_batch.to(device))
    test_preds[i * test_bs:(i + 1) * test_bs] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [ ]:
coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4


sample = pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)

In [ ]:
sample["diagnosis"].value_counts()